In [1]:
from datetime import datetime
import csv
import pandas as pd
import numpy as np

In [3]:
# dfs = []
# for x in range(ord('A'), ord('Z')+1):
#     dfs.append(pd.read_csv(f'storage/data-02-option_data.{chr(x)}.csv'))
# df = pd.concat(dfs)

df = pd.read_csv(f'data-02-option-data.csv')

# FILTERS
df = df[df.tradability == 'tradable']

# DROP COLUMNS
df.drop(['chain_id'], axis=1, inplace=True)
df.drop(['chain_symbol'], axis=1, inplace=True)
df.drop(['created_at'], axis=1, inplace=True)
df.drop(['high_fill_rate_buy_price'], axis=1, inplace=True)
df.drop(['high_fill_rate_sell_price'], axis=1, inplace=True)
df.drop(['id'], axis=1, inplace=True)
df.drop(['instrument'], axis=1, inplace=True)
df.drop(['instrument_id'], axis=1, inplace=True)
df.drop(['issue_date'], axis=1, inplace=True)
df.drop(['low_fill_rate_buy_price'], axis=1, inplace=True)
df.drop(['low_fill_rate_sell_price'], axis=1, inplace=True)
df.drop(['min_ticks'], axis=1, inplace=True)
df.drop(['occ_symbol'], axis=1, inplace=True)
df.drop(['rhs_tradability'], axis=1, inplace=True)
df.drop(['sellout_datetime'], axis=1, inplace=True)
df.drop(['state'], axis=1, inplace=True)
df.drop(['tradability'], axis=1, inplace=True)
df.drop(['updated_at'], axis=1, inplace=True)
df.drop(['url'], axis=1, inplace=True)

df = df[df.bid_size > 50]
df = df[df.mark_price > .30]
df = df[df.type == 'put']
df = df[df.volume > 50]
df = df[df.strike_price < 15.00]

df['percent_profit'] = df.mark_price / df.strike_price

today = datetime.now()
df.expiration_date = pd.to_datetime(df.expiration_date)
df['days_held'] = df.expiration_date.apply(lambda x: (x - today).days)
df['annualized'] = (df.percent_profit / df.days_held) * 365

df = df[df.annualized > 1.0]
df.shape

(72, 30)

In [4]:
df.head(n=40)

,expiration_date,strike_price,type,adjusted_mark_price,ask_price,ask_size,bid_price,bid_size,break_even_price,high_price,...,chance_of_profit_short,delta,gamma,implied_volatility,rho,theta,vega,percent_profit,days_held,annualized
1250,2021-07-16,5.0,put,0.78,0.80,14,0.75,71,4.22,1.00,...,0.645719,-0.236188,0.077318,2.006321,-0.002189,-0.017672,0.006122,0.155000,34,1.663971
2095,2021-07-16,12.5,put,1.23,1.30,55,1.15,57,11.27,1.45,...,0.667733,-0.301151,0.066794,1.194683,-0.005225,-0.026125,0.015202,0.098000,34,1.052059
3169,2021-07-16,3.0,put,0.40,0.45,147,0.35,146,2.60,0.43,...,0.625685,-0.352787,0.282711,1.322872,-0.001461,-0.007021,0.003690,0.133333,34,1.431373
4063,2021-07-09,12.5,put,1.01,1.04,65,0.97,116,11.49,1.22,...,0.685859,-0.290955,0.074304,1.180539,-0.003901,-0.028502,0.013399,0.080400,27,1.086889
4077,2021-06-18,13.0,put,0.38,0.38,2,0.37,143,12.62,0.65,...,0.749150,-0.262197,0.147720,1.145508,-0.000755,-0.053374,0.006272,0.028846,6,1.754808
4122,2021-06-25,13.0,put,0.85,0.87,66,0.83,102,12.15,1.12,...,0.687410,-0.318695,0.100066,1.297308,-0.002019,-0.046366,0.009818,0.065385,13,1.835799
4186,2021-06-25,12.0,put,0.47,0.48,175,0.45,52,11.53,0.60,...,0.764620,-0.210830,0.083073,1.264478,-0.001301,-0.036571,0.007944,0.038750,13,1.087981
4307,2021-07-16,4.0,put,0.70,0.75,344,0.65,130,3.30,0.75,...,0.631178,-0.238039,0.090184,2.160258,-0.001832,-0.015283,0.004916,0.175000,34,1.878676
4317,2021-07-16,3.5,put,0.40,0.45,153,0.35,140,3.10,0.40,...,0.702587,-0.173683,0.081985,1.969787,-0.001232,-0.011552,0.004075,0.114286,34,1.226891
4323,2021-06-18,5.0,put,0.63,0.65,1,0.60,386,4.37,0.85,...,0.616606,-0.397734,0.214767,2.568891,-0.000494,-0.051482,0.002697,0.125000,6,7.604167


In [5]:
# expiration_date	strike_price	adjusted_mark_price	ask_price	ask_size	bid_price	bid_size	break_even_price	high_price	last_trade_price	last_trade_size	low_price	mark_price	open_interest	previous_close_date	previous_close_price	volume	symbol	chance_of_profit_long	chance_of_profit_short	delta	gamma	implied_volatility	rho	theta	vega	percent_profit	days_held	annualized			
columns = [
    'annualized',
    'expiration_date',
    'rating',
    'symbol',
    'strike_price',
    'mark_price',
]
df['rating'] = ''
df.to_csv('data-04-interesting.csv', header=True, index=False, quoting=csv.QUOTE_NONNUMERIC, columns=columns)
df.shape

(72, 31)